<a href="https://colab.research.google.com/github/shubhii0206/Adaptive-Graph-Pooling-for-Protein-Structure-Classification-Using-GCNs/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# D&D Binary Classification

In [ ]:
!pip install torch torchvision torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.4 MB/s eta 0:00:00


In [ ]:
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
dataset = TUDataset(root='data/DD', name='DD')

Processing...
Done!


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling, global_mean_pool
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.nn import DenseGCNConv as GCNConv, dense_diff_pool as DiffPool

In [ ]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=3, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=3, shuffle=False)

In [ ]:
for data in train_loader:
  print(data.x.shape)

torch.Size([619, 89])
torch.Size([521, 89])
torch.Size([707, 89])
torch.Size([825, 89])
torch.Size([743, 89])
torch.Size([862, 89])
torch.Size([1107, 89])
torch.Size([806, 89])
torch.Size([1065, 89])
torch.Size([430, 89])
torch.Size([797, 89])
torch.Size([467, 89])
torch.Size([1100, 89])
torch.Size([1161, 89])
torch.Size([823, 89])
torch.Size([1214, 89])
torch.Size([961, 89])
torch.Size([538, 89])
torch.Size([796, 89])
torch.Size([768, 89])
torch.Size([580, 89])
torch.Size([730, 89])
torch.Size([455, 89])
torch.Size([924, 89])
torch.Size([602, 89])
torch.Size([730, 89])
torch.Size([777, 89])
torch.Size([895, 89])
torch.Size([577, 89])
torch.Size([685, 89])
torch.Size([711, 89])
torch.Size([643, 89])
torch.Size([968, 89])
torch.Size([723, 89])
torch.Size([591, 89])
torch.Size([483, 89])
torch.Size([1415, 89])
torch.Size([1465, 89])
torch.Size([1693, 89])
torch.Size([845, 89])
torch.Size([576, 89])
torch.Size([642, 89])
torch.Size([651, 89])
torch.Size([675, 89])
torch.Size([491, 89])
to

# Model Architecture

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling, global_mean_pool

class DiffPoolLayer(torch.nn.Module):
  def __init__(self, in_channels, hidden_channels, num_clusters):
    super(DiffPoolLayer, self).__init__()
    # GNN for learning node embeddings
    self.gnn_embed = GCNConv(in_channels, hidden_channels)
    # GNN for learning cluster assignments
    self.gnn_assign = GCNConv(in_channels, num_clusters)

  def forward(self, x, edge_index, batch):
    # Learn node embeddings
    h = F.relu(self.gnn_embed(x, edge_index))
    # Learn assignment matrix (cluster assignments)
    s = torch.softmax(self.gnn_assign(x, edge_index), dim=-1)

    # Pooling: Aggregate node features into clusters
    x_pooled = torch.matmul(s.T, h)  # (num_clusters, hidden_channels)

    # New adjacency matrix after coarsening
    adj_pooled = torch.matmul(s.T, s)  # (num_clusters, num_clusters)

    # Reconstruct edge_index from pooled adjacency matrix
    edge_index_pooled = (adj_pooled > 0).nonzero(as_tuple=False).T

    # Batch update: assume each cluster belongs to the same graph as the original nodes
    num_graphs = batch.max().item() + 1
    clusters_per_graph = max(1, s.shape[1] // num_graphs)

    batch_pooled = torch.repeat_interleave(torch.arange(num_graphs), clusters_per_graph)[:s.shape[1]]

    #batch_pooled = torch.repeat_interleave(torch.arange(batch.max() + 1), s.shape[1] // (batch.max() + 1))
    #assert x.size(0) == batch.max().item() + 1, "Batch size mismatch!"
    if (batch_pooled.size() == 2):
      batch_pooled += 1
    return x_pooled, edge_index_pooled, batch_pooled

class GraphClassificationModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, m1, m2, k1=0.2, k2=0.1):
      super(GraphClassificationModel, self).__init__()

      # GNN Layers
      self.gnn1 = GCNConv(input_dim, hidden_dim)
      self.gnn2 = GCNConv(hidden_dim, hidden_dim)

      # gPool Layer 1
      self.pool1 = TopKPooling(hidden_dim, ratio=k1)

      # DiffPool Layer 1
      self.diffpool1 = DiffPoolLayer(hidden_dim, hidden_dim, m1)

      self.gnn3 = GCNConv(hidden_dim, hidden_dim)
      self.gnn4 = GCNConv(hidden_dim, hidden_dim)

      # gPool Layer 2
      self.pool2 = TopKPooling(hidden_dim, ratio=k2)

      # DiffPool Layer 2
      self.diffpool2 = DiffPoolLayer(hidden_dim, hidden_dim, m2)

      # Final classification layer
      self.fc = torch.nn.Linear(hidden_dim, num_classes)

    def forward(self, data):
      x, edge_index, batch = data.x, data.edge_index, data.batch

      # Initial GNN Layers
      x = F.relu(self.gnn1(x, edge_index))
      x = F.relu(self.gnn2(x, edge_index))
      # print(f"Starting: x size = {x.size()}, batch size = {batch.size()}")
      # print("-------------------------------------")
      # gPool Layer 1
      # Modified: Unpack only the necessary values
      x, edge_index, _, batch, perm, score = self.pool1(x, edge_index, None, batch)
      # print(f"After gpool1: x size = {x.size()}, batch size = {batch.size()}")
      # print("-------------------------------------")
      # print(f"batch size: {batch.size()}, perm size: {perm.size()}")
      # print(f"max(perm): {perm.max()}, len(batch): {len(batch)}")
      if perm.size(0) > 0:
        perm = perm[perm < batch.size(0)]
        batch = batch[perm]  # Update batch after pooling

      # DiffPool Layer 1
      x, edge_index, batch = self.diffpool1(x, edge_index, batch)
      # print(f"After diffpool1: x size = {x.size()}, batch size = {batch.size()}")
      # print("-------------------------------------")

      # Additional GNN Layers
      x = F.relu(self.gnn3(x, edge_index))
      x = F.relu(self.gnn4(x, edge_index))

      # gPool Layer 2
      # Modified: Unpack only the necessary values
      #print(f"After DiffPool: x size = {x.size()}, batch size = {batch.size()}")

      x, edge_index, _, batch, perm, score = self.pool2(x, edge_index, None, batch)
      # print(f"After gpool2: x size = {x.size()}, batch size = {batch.size()}")
      # print("-------------------------------------")
      # print(f"batch size: {batch.size()}, perm size: {perm.size()}")
      # print(f"max(perm): {perm.max()}, len(batch): {len(batch)}")
      # perm = perm[perm < batch.size(0)]
      # batch = batch[perm]  # Update batch after pooling

      # # DiffPool Layer 2
      # x, edge_index, batch = self.diffpool2(x, edge_index, batch)

      # # Global Pooling for graph-level embedding
      # x = global_mean_pool(x, batch)

       # Check for empty graphs after pooling
      if batch.size(0) == 0:
          # Handle empty graphs (e.g., skip or assign a default output)
          # For example, you could return a zero vector for these graphs:
          print("graph is empty")
          x = torch.zeros(1, self.hidden_dim, device=x.device)
          batch = torch.zeros(1, dtype=torch.long, device=x.device)

      else:

          perm = perm[perm < batch.size(0)]
          batch = batch[perm]  # Update batch after pooling

      # DiffPool Layer 2
      x, edge_index, batch = self.diffpool2(x, edge_index, batch)
      # print(f"After diffpool2: x size = {x.size()}, batch size = {batch.size()}")
      # print("-------------------------------------")



      # Global Pooling for graph-level embedding
      x = global_mean_pool(x, batch)
      # print(f"After globalmean: x size = {x.size()}, batch size = {batch.size()}")

      # Final Classification Head
      x = self.fc(x)
      #print(f"After fc: x size = {x.size()}, batch size = {batch.size()}")
      #print(F.log_softmax(x, dim=1))
      return F.log_softmax(x, dim=1)


# Training and Evaluation Functions

In [ ]:
def train(model, loader, optimizer, criterion):
  model.train()
  total_loss = 0
  for data in loader:
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out, data.y)

    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  return total_loss / len(loader)

def test(model, loader):
  model.eval()
  correct = 0
  for data in loader:
    out = model(data)

    pred = out.argmax(dim=1)
    correct += int((pred == data.y).sum())
  return correct / len(loader.dataset)

# Experimenting with Different Hyperparameters

In [ ]:
# Initialize the model with specific hyperparameters
model = GraphClassificationModel(
    input_dim=dataset.num_node_features,
    hidden_dim=64,  # Adjust as needed
    num_classes=dataset.num_classes,
    m1=6,  # First DiffPool with 6 clusters
    m2=3,  # Second DiffPool with 3 clusters
    k1=0.9,  # 90% top nodes kept in first gPool layer
    k2=0.8   # 80% top nodes kept in second gPool layer
)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(1, 31):
  train_loss = train(model, train_loader, optimizer, criterion)
  train_acc = test(model, train_loader)
  #test_acc = test(model, test_loader)
  print(f'Epoch: {epoch}, Train Loss: {train_loss:.4f}, Test Accuracy: {train_acc:.4f}')

Epoch: 1, Train Loss: 0.7154, Test Accuracy: 0.5807
Epoch: 2, Train Loss: 0.6815, Test Accuracy: 0.5807
Epoch: 3, Train Loss: 0.6822, Test Accuracy: 0.5807
Epoch: 4, Train Loss: 0.6820, Test Accuracy: 0.5807
Epoch: 5, Train Loss: 0.6819, Test Accuracy: 0.5807
Epoch: 6, Train Loss: 0.6819, Test Accuracy: 0.5807
Epoch: 7, Train Loss: 0.6825, Test Accuracy: 0.5807
Epoch: 8, Train Loss: 0.6820, Test Accuracy: 0.5807
Epoch: 9, Train Loss: 0.6819, Test Accuracy: 0.5807
Epoch: 10, Train Loss: 0.6823, Test Accuracy: 0.5807
Epoch: 11, Train Loss: 0.6818, Test Accuracy: 0.5807
Epoch: 12, Train Loss: 0.6821, Test Accuracy: 0.5807
Epoch: 13, Train Loss: 0.6823, Test Accuracy: 0.5807
Epoch: 14, Train Loss: 0.6798, Test Accuracy: 0.5807
Epoch: 15, Train Loss: 0.6825, Test Accuracy: 0.5807
Epoch: 16, Train Loss: 0.6822, Test Accuracy: 0.5807
Epoch: 17, Train Loss: 0.6834, Test Accuracy: 0.5807
Epoch: 18, Train Loss: 0.6815, Test Accuracy: 0.5807
Epoch: 19, Train Loss: 0.6818, Test Accuracy: 0.5807
Ep

In [ ]:
for batch in train_loader:
  print(f"Number of graphs in batch: {batch.num_graphs}")


In [ ]:
input_dim = dataset.num_node_features
hidden_dim = 64
num_classes = 2

k_values = [0.9, 0.8, 0.6]
results = {}

for k1 in k_values:
  for k2 in k_values:
    if k1 == k2:
      continue
    model = GraphClassificationModel(input_dim, hidden_dim, num_classes, m1=6, m2=3, k1=k1, k2=k2)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = torch.nn.CrossEntropyLoss()
    print(f"k1 = {k1} and k2 = {k2}")
    for epoch in range(1, 11):
      train_loss = train(model, train_loader, optimizer, criterion)
      test_acc = test(model, train_loader)
      print(f'Epoch: {epoch}, Train Loss: {train_loss:.4f}, Test Accuracy: {test_acc:.4f}')
      results[(k1, k2)] = test_acc
    print(f'Final Test Accuracy with k1={k1}, k2={k2}: {test_acc:.4f}\n')

k1 = 0.9 and k2 = 0.8
Epoch: 1, Train Loss: 0.6984, Test Accuracy: 0.5786
Epoch: 2, Train Loss: 0.6827, Test Accuracy: 0.5786
Epoch: 3, Train Loss: 0.6827, Test Accuracy: 0.5786
Epoch: 4, Train Loss: 0.6833, Test Accuracy: 0.5786
Epoch: 5, Train Loss: 0.6811, Test Accuracy: 0.5786
Epoch: 6, Train Loss: 0.6823, Test Accuracy: 0.5786
Epoch: 7, Train Loss: 0.6828, Test Accuracy: 0.5786
Epoch: 8, Train Loss: 0.6829, Test Accuracy: 0.5786
Epoch: 9, Train Loss: 0.6826, Test Accuracy: 0.5786
Epoch: 10, Train Loss: 0.6841, Test Accuracy: 0.5786
Final Test Accuracy with k1=0.9, k2=0.8: 0.5786

k1 = 0.9 and k2 = 0.6
Epoch: 1, Train Loss: 0.7319, Test Accuracy: 0.5786
Epoch: 2, Train Loss: 0.6831, Test Accuracy: 0.5786
Epoch: 3, Train Loss: 0.6828, Test Accuracy: 0.5786
Epoch: 4, Train Loss: 0.6827, Test Accuracy: 0.5786
Epoch: 5, Train Loss: 0.6822, Test Accuracy: 0.5786
Epoch: 6, Train Loss: 0.6835, Test Accuracy: 0.5786
Epoch: 7, Train Loss: 0.6826, Test Accuracy: 0.5786
Epoch: 8, Train Loss: 

In [ ]:
import matplotlib.pyplot as plt

def plot_metrics(metrics, k_values):
  for k1, metric in metrics.items():
    plt.plot(metric['epoch'], metric['accuracy'], label=f'k1={k1}')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.title('Test Accuracy over Epochs')
  plt.legend()
  plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Convert results into a 2D matrix for plotting
accuracy_matrix = np.array([
    [results[(k1, k2)] for k2 in k_values]
    for k1 in k_values
])

# Create a heatmap using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(accuracy_matrix, annot=True, cmap="YlGnBu", xticklabels=k_values, yticklabels=k_values)

plt.title("k1, k2 vs Accuracy")
plt.xlabel("k2 Values")
plt.ylabel("k1 Values")
plt.show()


KeyError: (0.9, 0.9)

# ENZYME dataset

In [ ]:
# Load the ENZYMES dataset
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')
# Print dataset information
print(f"Number of graphs: {len(dataset)}")
print(f"Number of classes: {dataset.num_classes}")
print(f"Number of node features: {dataset.num_node_features}")

# Access a single graph
data = dataset[0]  # Get the first graph example
print(data)

Processing...


Number of graphs: 600
Number of classes: 6
Number of node features: 3
Data(edge_index=[2, 168], x=[37, 3], y=[1])


Done!


In [ ]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=3, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=3, shuffle=False)

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling, global_mean_pool

class DiffPoolLayer(torch.nn.Module):
  def __init__(self, in_channels, hidden_channels, num_clusters):
    super(DiffPoolLayer, self).__init__()
    # GNN for learning node embeddings
    self.gnn_embed = GCNConv(in_channels, hidden_channels)
    # GNN for learning cluster assignments
    self.gnn_assign = GCNConv(in_channels, num_clusters)

  def forward(self, x, edge_index, batch):
    # Learn node embeddings
    h = F.relu(self.gnn_embed(x, edge_index))
    # Learn assignment matrix (cluster assignments)
    s = torch.softmax(self.gnn_assign(x, edge_index), dim=-1)

    # Pooling: Aggregate node features into clusters
    x_pooled = torch.matmul(s.T, h)  # (num_clusters, hidden_channels)

    # New adjacency matrix after coarsening
    adj_pooled = torch.matmul(s.T, s)  # (num_clusters, num_clusters)

    # Reconstruct edge_index from pooled adjacency matrix
    edge_index_pooled = (adj_pooled > 0).nonzero(as_tuple=False).T

    # Batch update: assume each cluster belongs to the same graph as the original nodes
    num_graphs = batch.max().item() + 1
    clusters_per_graph = max(1, s.shape[1] // num_graphs)

    batch_pooled = torch.repeat_interleave(torch.arange(num_graphs), clusters_per_graph)[:s.shape[1]]

    #batch_pooled = torch.repeat_interleave(torch.arange(batch.max() + 1), s.shape[1] // (batch.max() + 1))
    #assert x.size(0) == batch.max().item() + 1, "Batch size mismatch!"
    if (batch_pooled.size() == 2):
      batch_pooled += 1
    return x_pooled, edge_index_pooled, batch_pooled

class GraphClassificationModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, m1, m2, k1=0.2, k2=0.1):
      super(GraphClassificationModel, self).__init__()

      # GNN Layers
      self.gnn1 = GCNConv(input_dim, hidden_dim)
      self.gnn2 = GCNConv(hidden_dim, hidden_dim)

      # gPool Layer 1
      self.pool1 = TopKPooling(hidden_dim, ratio=k1)

      # DiffPool Layer 1
      self.diffpool1 = DiffPoolLayer(hidden_dim, hidden_dim, m1)

      self.gnn3 = GCNConv(hidden_dim, hidden_dim)
      self.gnn4 = GCNConv(hidden_dim, hidden_dim)

      # gPool Layer 2
      self.pool2 = TopKPooling(hidden_dim, ratio=k2)

      # DiffPool Layer 2
      self.diffpool2 = DiffPoolLayer(hidden_dim, hidden_dim, m2)

      # Final classification layer
      self.fc = torch.nn.Linear(hidden_dim, num_classes)

    def forward(self, data):
      x, edge_index, batch = data.x, data.edge_index, data.batch

      # Initial GNN Layers
      x = F.relu(self.gnn1(x, edge_index))
      x = F.relu(self.gnn2(x, edge_index))
      # print(f"Starting: x size = {x.size()}, batch size = {batch.size()}")
      # print("-------------------------------------")
      # gPool Layer 1
      # Modified: Unpack only the necessary values
      x, edge_index, _, batch, perm, score = self.pool1(x, edge_index, None, batch)
      # print(f"After gpool1: x size = {x.size()}, batch size = {batch.size()}")
      # print("-------------------------------------")
      # print(f"batch size: {batch.size()}, perm size: {perm.size()}")
      # print(f"max(perm): {perm.max()}, len(batch): {len(batch)}")
      if perm.size(0) > 0:
        perm = perm[perm < batch.size(0)]
        batch = batch[perm]  # Update batch after pooling

      # DiffPool Layer 1
      x, edge_index, batch = self.diffpool1(x, edge_index, batch)
      # print(f"After diffpool1: x size = {x.size()}, batch size = {batch.size()}")
      # print("-------------------------------------")

      # Additional GNN Layers
      x = F.relu(self.gnn3(x, edge_index))
      x = F.relu(self.gnn4(x, edge_index))

      # gPool Layer 2
      # Modified: Unpack only the necessary values
      #print(f"After DiffPool: x size = {x.size()}, batch size = {batch.size()}")

      x, edge_index, _, batch, perm, score = self.pool2(x, edge_index, None, batch)
      # print(f"After gpool2: x size = {x.size()}, batch size = {batch.size()}")
      # print("-------------------------------------")
      # print(f"batch size: {batch.size()}, perm size: {perm.size()}")
      # print(f"max(perm): {perm.max()}, len(batch): {len(batch)}")
      # perm = perm[perm < batch.size(0)]
      # batch = batch[perm]  # Update batch after pooling

      # # DiffPool Layer 2
      # x, edge_index, batch = self.diffpool2(x, edge_index, batch)

      # # Global Pooling for graph-level embedding
      # x = global_mean_pool(x, batch)

       # Check for empty graphs after pooling
      if batch.size(0) == 0:
          # Handle empty graphs (e.g., skip or assign a default output)
          # For example, you could return a zero vector for these graphs:
          print("graph is empty")
          x = torch.zeros(1, self.hidden_dim, device=x.device)
          batch = torch.zeros(1, dtype=torch.long, device=x.device)

      else:

          perm = perm[perm < batch.size(0)]
          batch = batch[perm]  # Update batch after pooling

      # DiffPool Layer 2
      x, edge_index, batch = self.diffpool2(x, edge_index, batch)
      # print(f"After diffpool2: x size = {x.size()}, batch size = {batch.size()}")
      # print("-------------------------------------")



      # Global Pooling for graph-level embedding
      x = global_mean_pool(x, batch)
      # print(f"After globalmean: x size = {x.size()}, batch size = {batch.size()}")

      # Final Classification Head
      x = self.fc(x)
      #print(f"After fc: x size = {x.size()}, batch size = {batch.size()}")
      #print(F.log_softmax(x, dim=1))
      return F.log_softmax(x, dim=1)


In [ ]:
# Initialize the model with specific hyperparameters
model = GraphClassificationModel(
    input_dim=dataset.num_node_features,
    hidden_dim=64,  # Adjust as needed
    num_classes=dataset.num_classes, # 6
    m1=6,  # First DiffPool with 6 clusters
    m2=3,  # Second DiffPool with 3 clusters
    k1=0.9,  # 90% top nodes kept in first gPool layer
    k2=0.8   # 80% top nodes kept in second gPool layer
)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(1, 31):
  train_loss = train(model, train_loader, optimizer, criterion)
  train_acc = test(model, train_loader)
  #test_acc = test(model, test_loader)
  print(f'Epoch: {epoch}, Train Loss: {train_loss:.4f}, Test Accuracy: {train_acc:.4f}')

Epoch: 1, Train Loss: 1.8105, Test Accuracy: 0.1771
Epoch: 2, Train Loss: 1.7975, Test Accuracy: 0.1771
Epoch: 3, Train Loss: 1.7959, Test Accuracy: 0.1771
Epoch: 4, Train Loss: 1.7961, Test Accuracy: 0.1771
Epoch: 5, Train Loss: 1.7955, Test Accuracy: 0.1771
Epoch: 6, Train Loss: 1.7969, Test Accuracy: 0.1771
Epoch: 7, Train Loss: 1.7955, Test Accuracy: 0.1771
Epoch: 8, Train Loss: 1.7976, Test Accuracy: 0.1771
Epoch: 9, Train Loss: 1.7962, Test Accuracy: 0.1667
Epoch: 10, Train Loss: 1.7955, Test Accuracy: 0.1771
Epoch: 11, Train Loss: 1.7966, Test Accuracy: 0.1771
Epoch: 12, Train Loss: 1.7974, Test Accuracy: 0.1771
Epoch: 13, Train Loss: 1.7959, Test Accuracy: 0.1771
Epoch: 14, Train Loss: 1.7959, Test Accuracy: 0.1771
Epoch: 15, Train Loss: 1.7968, Test Accuracy: 0.1771
Epoch: 16, Train Loss: 1.7983, Test Accuracy: 0.1771
Epoch: 17, Train Loss: 1.7967, Test Accuracy: 0.1771
Epoch: 18, Train Loss: 1.7951, Test Accuracy: 0.1771
Epoch: 19, Train Loss: 1.7960, Test Accuracy: 0.1771
Ep